In [3]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [8]:
def extract_columns(pathname, output_pathname, min_size=16):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.nii.gz"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.extruded-overlay-NS.nii.gz"))
    print(len(data_paths),len(data_overlay_paths))
    img = itk.GetArrayFromImage(itk.imread(data_paths[0], itk.F))
    print(img.shape)
    num_overlays = len(data_overlay_paths)
    for i in range(num_overlays):
        img = itk.GetArrayFromImage(itk.imread(data_paths[i], itk.F))
        ovrimg = itk.GetArrayFromImage(itk.imread(data_overlay_paths[i], itk.UC))
        print("Processing", data_overlay_paths[i])
        
        # Eliminate ribs and map sliding to 2
        ovrimg = np.where(ovrimg==3,2,ovrimg)

        nos = 0
        sli = 0
        c = 0
        while c<ovrimg.shape[2]:
            nos = np.count_nonzero(ovrimg[:,:,c]==1)
            sli = np.count_nonzero(ovrimg[:,:,c]==2)
            start_c = c
            end_c = c
            val_c = 0
            name_c = ''
            if nos>0 and sli==0:
                val_c = 1
                name_c = "N"
            elif sli>0 and nos==0:
                val_c = 2
                name_c = "S"
            else:
                c += 1
                continue
            start_c = c
            end_c = c
            c += 1
            count = 0
            if c<ovrimg.shape[2]:
                count = np.count_nonzero(ovrimg[:,:,c]==val_c)
                while count>0 and c<ovrimg.shape[2]-1:
                    c += 1
                    count = np.count_nonzero(ovrimg[:,:,c]==val_c)
            end_c = c
            if end_c-start_c>=min_size:
                y_start = ovrimg.shape[1]
                y_end = 0
                for c in range(start_c,end_c):
                    for z in range(ovrimg.shape[0]):
                        pos = np.argwhere(ovrimg[z,:,c]==val_c)
                        if len(pos)>0:
                            y_start = min(y_start,pos[0][0])
                            y_end = max(y_end,pos[-1][0])
                inshape = list(img.shape)
                outshape = list(img.shape)
                outshape[1] = 224
                outshape[2] = 128
                print(y_start, y_end)
                
                y_start = max(0, y_start - (outshape[1]//5))
                y_end = min(inshape[1], y_start+outshape[1])
                if y_end-y_start<outshape[1]:
                    y_start = y_end-outshape[1]
                
                start_c = max(0, (start_c+end_c)//2 - (outshape[2]//2))
                end_c = min(inshape[2], start_c+outshape[2])
                if end_c-start_c<outshape[2]:
                    start_c = end_c-outshape[2]
                    
                outimg = img[:,y_start:y_end,start_c:end_c]
                outovr = ovrimg[:,y_start:y_end,start_c:end_c]

                (filepath,filename) = os.path.split(data_overlay_paths[i])
                
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.roi.nii.gz'
                itkimg = itk.GetImageFromArray(outimg.astype(np.short))
                itk.imwrite(itkimg, new_name, compression=True)
                
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.roi.extruded-overlay-NS.nii.gz'
                itkimg = itk.GetImageFromArray(outovr.astype(np.short))
                itk.imwrite(itkimg, new_name, compression=True)
                
                outovr = np.where(outovr==1,0,outovr)
                outovr = np.where(outovr==2,1,outovr)
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.roi.extruded-overlay-S.nii.gz'
                itkimg = itk.GetImageFromArray(outovr.astype(np.short))
                itk.imwrite(itkimg, new_name, compression=True)

In [9]:
output_pathname = "../../Data/VFoldData/ROIData"

pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
extract_columns(pathname, output_pathname, 16)
pathname = "../../Data/VFoldData/BAMC-PTXSliding"
extract_columns(pathname, output_pathname, 16)

30 30
(68, 320, 320)
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_Image_262499828648_clean.extruded-overlay-NS.nii.gz
206 319
210 319
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_image_267456908021_clean.extruded-overlay-NS.nii.gz
224 319
203 319
205 319
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1083297968960_clean.extruded-overlay-NS.nii.gz
156 264
90 305
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1087766719219_clean.extruded-overlay-NS.nii.gz
153 266
89 302
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/027ns_image_4641643404894_CLEAN.extruded-overlay-NS.nii.gz
29 296
35 266
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/027ns_image_4743880599022_clean.extruded-overlay-NS.nii.gz
51 319
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/035ns_image_1394469579519_clean.extruded-overlay-NS.n

Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/228s_iimage_3384882513134_clean.extruded-overlay-NS.nii.gz
174 319
201 302
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/236s_iimage_1139765223418_CLEAN.extruded-overlay-NS.nii.gz
89 241
86 316
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/236s_iimage_1327616672148_clean.extruded-overlay-NS.nii.gz
63 319
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/237s_iimage_24164968068436_CLEAN.extruded-overlay-NS.nii.gz
202 319
178 235
